# Data Cleasing Task

## 1. Importing and Inspecting

In [16]:
# Import packages
import pandas as pd

In [17]:
# Read in raw data
data_raw = pd.read_excel('Data/Input/messy_phol.xlsx')

data_raw.head()

,region,date,name,comment
0,ACT,2014-01-01,New Years Day,None
1,ACT,2014-01-27,Australia Day Sub,None
2,ACT,2014-03-10,Canberra Day -RD,None
3,ACT,2014-04-18,Good Friday,None
4,ACT,2014-04-19,Easter Saturday,None


In [18]:
data_raw.dtypes

region             object
date       datetime64[ns]
name               object
comment            object
dtype: object

`date` field is already in proper date type.

Data at index 470 and 662 has missing values.

In [19]:
print(data_raw[data_raw.isnull().any(axis=1)])

    region       date                    name comment
470    NaN 2019-04-25               ANZAC Day    None
662     WA        NaT  Queens Birthday WA -RD    None


The names of public holidays are very messy.

In [20]:
print(data_raw['name'].value_counts())

Christmas Day                                     63
Easter Monday                                     62
Good Friday                                       58
ANZAC Day                                         50
New Years Day                                     47
                                                  ..
Boxing Day / Proclamation Day (additional day)     1
Australia Day 26th                                 1
good Friday - type c                               1
Australia Day SUB                                  1
Easter MON                                         1
Name: name, Length: 72, dtype: int64


Here I take a subset of the data during the in-scope period.  
The rest of the data can be used to help determine names and dates of holidays.

In [21]:
data_inscope = data_raw[(data_raw['date'] > '2019-01-01') & (data_raw['date'] < '2021-12-31')]

## 2. Data Cleansing

### 2.1 Missing Data

There is data missing at index 470 and 662, and the data is roughly ordered by state. Therefore:
1. We inspect the data points around index 470 and 662
2. 